In [38]:
import os

import numpy as np
import pandas as pd
import pickle

from tensorflow.python.ops.gen_experimental_dataset_ops import experimental_latency_stats_dataset_eager_fallback

with open(f"arxiv-abstracts-2021/pickle.pkl", "rb") as f:
    data = pickle.load(f)

In [39]:
documents = data["title"] + " " + data["abstract"]
labels = data["categories"]
del data

In [40]:
full_name_translator = {}
full_name_translator["cs"] = "Computer Science"
full_name_translator["cs.AI"] = "Artificial Intelligence"
full_name_translator["cs.AR"] = "Hardware Architecture"
full_name_translator["cs.CC"] = "Computational Complexity"
full_name_translator["cs.CE"] = "Computational Engineering, Finance, and Science"
full_name_translator["cs.CG"] = "Computational Geometry"
full_name_translator["cs.CL"] = "Computation and Language"
full_name_translator["cs.CR"] = "Cryptography and Security"
full_name_translator["cs.CV"] = "Computer Vision and Pattern Recognition"
full_name_translator["cs.CY"] = "Computers and Society"
full_name_translator["cs.DB"] = "Databases"
full_name_translator["cs.DC"] = "Distributed, Parallel, and Cluster Computing"
full_name_translator["cs.DL"] = "Digital Libraries"
full_name_translator["cs.DM"] = "Discrete Mathematics"
full_name_translator["cs.DS"] = "Data Structures and Algorithms"
full_name_translator["cs.ET"] = "Emerging Technologies"
full_name_translator["cs.FL"] = "Formal Languages and Automata Theory"
full_name_translator["cs.GL"] = "General Literature"
full_name_translator["cs.GR"] = "Graphics"
full_name_translator["cs.GT"] = "Computer Science and Game Theory"
full_name_translator["cs.HC"] = "Human-Computer Interaction"
full_name_translator["cs.IR"] = "Information Retrieval"
full_name_translator["cs.IT"] = "Information Theory"
full_name_translator["cs.LG"] = "Machine Learning"
full_name_translator["cs.LO"] = "Logic in Computer Science"
full_name_translator["cs.MA"] = "Multiagent Systems"
full_name_translator["cs.MM"] = "Multimedia"
full_name_translator["cs.MS"] = "Mathematical Software"
full_name_translator["cs.NA"] = "Numerical Analysis"
full_name_translator["cs.NE"] = "Neural and Evolutionary Computing"
full_name_translator["cs.NI"] = "Networking and Internet Architecture"
full_name_translator["cs.OH"] = "Other Computer Science"
full_name_translator["cs.OS"] = "Operating Systems"
full_name_translator["cs.PF"] = "Performance"
full_name_translator["cs.PL"] = "Programming Languages"
full_name_translator["cs.RO"] = "Robotics"
full_name_translator["cs.SC"] = "Symbolic Computation"
full_name_translator["cs.SD"] = "Sound"
full_name_translator["cs.SE"] = "Software Engineering"
full_name_translator["cs.SI"] = "Social and Information Networks"
full_name_translator["cs.SY"] = "Systems and Control"

full_name_translator["econ"] = "Economics"
full_name_translator["econ.EM"] = "Econometrics"
full_name_translator["econ.GN"] = "General Economics"
full_name_translator["econ.TH"] = "Theoretical Economics"

full_name_translator["eess"] = "Electrical Engineering and Systems Science"
full_name_translator["eess.AS"] = "Audio and Speech Processing"
full_name_translator["eess.IV"] = "Image and Video Processing"
full_name_translator["eess.SP"] = "Signal Processing"
full_name_translator["eess.SY"] = "Systems and Control"

full_name_translator["math"] = "Mathematics"
full_name_translator["math.AC"] = "Commutative Algebra"
full_name_translator["math.AG"] = "Algebraic Geometry"
full_name_translator["math.AP"] = "Analysis of PDEs"
full_name_translator["math.AT"] = "Algebraic Topology"
full_name_translator["math.CA"] = "Classical Analysis and ODEs"
full_name_translator["math.CO"] = "Combinatorics"
full_name_translator["math.CT"] = "Category Theory"
full_name_translator["math.CV"] = "Complex Variables"
full_name_translator["math.DG"] = "Differential Geometry"
full_name_translator["math.DS"] = "Dynamical Systems"
full_name_translator["math.FA"] = "Functional Analysis"
full_name_translator["math.GM"] = "General Mathematics"
full_name_translator["math.GN"] = "General Topology"
full_name_translator["math.GR"] = "Group Theory"
full_name_translator["math.GT"] = "Geometric Topology"
full_name_translator["math.HO"] = "History and Overview"
full_name_translator["math.IT"] = "Information Theory"
full_name_translator["math.KT"] = "K-Theory and Homology"
full_name_translator["math.LO"] = "Logic"
full_name_translator["math.MG"] = "Metric Geometry"
full_name_translator["math.MP"] = "Mathematical Physics"
full_name_translator["math.NA"] = "Numerical Analysis"
full_name_translator["math.NT"] = "Number Theory"
full_name_translator["math.OA"] = "Operator Algebras"
full_name_translator["math.OC"] = "Optimization and Control"
full_name_translator["math.PR"] = "Probability"
full_name_translator["math.QA"] = "Quantum Algebra"
full_name_translator["math.RA"] = "Rings and Algebras"
full_name_translator["math.RT"] = "Representation Theory"
full_name_translator["math.SG"] = "Symplectic Geometry"
full_name_translator["math.SP"] = "Spectral Theory"
full_name_translator["math.ST"] = "Statistics Theory"

full_name_translator["astro-ph"] = "Astrophysics"
full_name_translator["astro-ph.CO"] = "Cosmology and Nongalactic Astrophysics"
full_name_translator["astro-ph.EP"] = "Earth and Planetary Astrophysics"
full_name_translator["astro-ph.GA"] = "Astrophysics of Galaxies"
full_name_translator["astro-ph.HE"] = "High Energy Astrophysical Phenomena"
full_name_translator["astro-ph.IM"] = "Instrumentation and Methods for Astrophysics"
full_name_translator["astro-ph.SR"] = "Solar and Stellar Astrophysics"

full_name_translator["cond-mat"] = "Condensed Matter"
full_name_translator["cond-mat.dis-nn"] = "Disordered Systems and Neural Networks"
full_name_translator["cond-mat.mes-hall"] = "Mesoscale and Nanoscale Physics"
full_name_translator["cond-mat.mtrl-sci"] = "Materials Science"
full_name_translator["cond-mat.other"] = "Other Condensed Matter"
full_name_translator["cond-mat.quant-gas"] = "Quantum Gases"
full_name_translator["cond-mat.soft"] = "Soft Condensed Matter"
full_name_translator["cond-mat.stat-mech"] = "Statistical Mechanics"
full_name_translator["cond-mat.str-el"] = "Strongly Correlated Electrons"
full_name_translator["cond-mat.supr-con"] = "Superconductivity"

full_name_translator["gr-qc"] = "General Relativity and Quantum Cosmology"
full_name_translator["hep-ex"] = "High Energy Physics - Experiment"
full_name_translator["hep-lat"] = "High Energy Physics - Lattice"
full_name_translator["hep-ph"] = "High Energy Physics - Phenomenology"
full_name_translator["hep-th"] = "High Energy Physics - Theory"
full_name_translator["math-ph"] = "Mathematical Physics"

full_name_translator["nlin"] = "Nonlinear Sciences"
full_name_translator["nlin.AO"] = "Adaptation and Self-Organizing Systems"
full_name_translator["nlin.CD"] = "Chaotic Dynamics"
full_name_translator["nlin.CG"] = "Cellular Automata and Lattice Gases"
full_name_translator["nlin.PS"] = "Pattern Formation and Solitons"
full_name_translator["nlin.SI"] = "Exactly Solvable and Integrable Systems"

full_name_translator["nucl-ex"] = "Nuclear Experiment"
full_name_translator["nucl-th"] = "Nuclear Theory"

full_name_translator["physics"] = "Physics"
full_name_translator["physics.acc-ph"] = "Accelerator Physics"
full_name_translator["physics.ao-ph"] = "Atmospheric and Oceanic Physics"
full_name_translator["physics.app-ph"] = "Applied Physics"
full_name_translator["physics.atm-clus"] = "Atomic and Molecular Clusters"
full_name_translator["physics.atom-ph"] = "Atomic Physics"
full_name_translator["physics.bio-ph"] = "Biological Physics"
full_name_translator["physics.chem-ph"] = "Chemical Physics"
full_name_translator["physics.class-ph"] = "Classical Physics"
full_name_translator["physics.comp-ph"] = "Computational Physics"
full_name_translator["physics.data-an"] = "Data Analysis, Statistics and Probability"
full_name_translator["physics.ed-ph"] = "Physics Education"
full_name_translator["physics.flu-dyn"] = "Fluid Dynamics"
full_name_translator["physics.gen-ph"] = "General Physics"
full_name_translator["physics.geo-ph"] = "Geophysics"
full_name_translator["physics.hist-ph"] = "History and Philosophy of Physics"
full_name_translator["physics.ins-det"] = "Instrumentation and Detectors"
full_name_translator["physics.med-ph"] = "Medical Physics"
full_name_translator["physics.optics"] = "Optics"
full_name_translator["physics.plasm-ph"] = "Plasma Physics"
full_name_translator["physics.pop-ph"] = "Popular Physics"
full_name_translator["physics.soc-ph"] = "Physics and Society"
full_name_translator["physics.space-ph"] = "Space Physics"

full_name_translator["quant-ph"] = "Quantum Physics"

full_name_translator["q-bio"] = "Quantitative Biology"
full_name_translator["q-bio.BM"] = "Biomolecules"
full_name_translator["q-bio.CB"] = "Cell Behavior"
full_name_translator["q-bio.GN"] = "Genomics"
full_name_translator["q-bio.MN"] = "Molecular Networks"
full_name_translator["q-bio.NC"] = "Neurons and Cognition"
full_name_translator["q-bio.OT"] = "Other Quantitative Biology"
full_name_translator["q-bio.PE"] = "Populations and Evolution"
full_name_translator["q-bio.QM"] = "Quantitative Methods"
full_name_translator["q-bio.SC"] = "Subcellular Processes"
full_name_translator["q-bio.TO"] = "Tissues and Organs"

full_name_translator["q-fin"] = "Quantitative Finance"
full_name_translator["q-fin.CP"] = "Computational Finance"
full_name_translator["q-fin.EC"] = "Economics"
full_name_translator["q-fin.GN"] = "General Finance"
full_name_translator["q-fin.MF"] = "Mathematical Finance"
full_name_translator["q-fin.PM"] = "Portfolio Management"
full_name_translator["q-fin.PR"] = "Pricing of Securities"
full_name_translator["q-fin.RM"] = "Risk Management"
full_name_translator["q-fin.ST"] = "Statistical Finance"
full_name_translator["q-fin.TR"] = "Trading and Market Microstructure"

full_name_translator["stat"] = "Statistics"
full_name_translator["stat.AP"] = "Applications"
full_name_translator["stat.CO"] = "Computation"
full_name_translator["stat.ME"] = "Methodology"
full_name_translator["stat.ML"] = "Machine Learning"
full_name_translator["stat.OT"] = "Other Statistics"
full_name_translator["stat.TH"] = "Statistics Theory"
# old labels
alias_map = {}
alias_map["acc-phys"] = "physics.acc-ph"
alias_map["adap-org"] = "nlin.AO"
alias_map["chao-dyn"] = "nlin.CD"
alias_map["patt-sol"] = "nlin.PS"
alias_map["dg-ga"] = "math.DG"
alias_map["solv-int"] = "nlin.SI"
alias_map["bayes-an"] = "physics.data-an"
alias_map["comp-gas"] = "nlin.CG"
alias_map["alg-geom"] = "math.AG"
alias_map["funct-an"] = "math.FA"
alias_map["q-alg"] = "math.QA"
alias_map["ao-sci"] = "physics.ao-ph"
alias_map["atom-ph"] = "physics.atom-ph"
alias_map["chem-ph"] = "physics.chem-ph"
alias_map["plasm-ph"] = "physics.plasm-ph"
alias_map["mtrl-th"] = "cond-mat.mtrl-sci"
alias_map["cmp-lg"] = "cs.CL"
alias_map["supr-con"] = "cond-mat.supr-con"


def to_full_name(key: str) -> str:
    return full_name_translator.get(alias_map.get(key, key))

In [41]:
def load_vectors(fname: str, word_set: set) -> dict:
    """
    Lädt nur die Vektoren für die benötigten Wörter.
    """
    import io
    vectors = {}
    with io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore') as fin:
        n, d = map(int, fin.readline().split())
        for line in fin:
            tokens = line.rstrip().split(' ')
            word = tokens[0]
            if word in word_set:
                vectors[word] = np.array([float(x) for x in tokens[1:]])
    return vectors

In [42]:
from embeddings.embedding_utils import simple_clean_list
import tqdm

In [43]:
cleaned_corpus = simple_clean_list(documents)
unique_words = set()

100%|██████████| 1999486/1999486 [15:06<00:00, 2206.70it/s] 


In [44]:

for doc in tqdm.tqdm(cleaned_corpus):
    unique_words.update(doc)


100%|██████████| 1999486/1999486 [03:58<00:00, 8373.34it/s] 


In [45]:
translator = load_vectors(r"..\embeddings\fasttext_data\wiki-news-300d-1M.vec", unique_words)

In [46]:
# how many unique words does the translator know?
unique_recognized_words = translator.keys()
print(f"Fasttext knows {len(unique_recognized_words)} words out of {len(unique_words)} total unique words. Thats {len(unique_recognized_words)/len(unique_words)*100:.0f}%")

Fasttext knows 105594 words out of 480706 total unique words. Thats 22%


In [47]:
missed_words = {word for word in unique_words if word not in translator.keys()}

In [48]:
from spellchecker import SpellChecker
import nltk
from nltk.corpus import words

# Load English words
nltk.download('words')
english_vocab = set(words.words())

# Load spellchecker
spell = SpellChecker()

# what words are known by default
real_words_nltk = {w for w in tqdm.tqdm(missed_words) if w in english_vocab}
real_words_spell = set(spell.known(missed_words))
real_words = real_words_nltk | real_words_spell

missed_words_uncategorized = missed_words - real_words

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\belas\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
100%|██████████| 375112/375112 [00:00<00:00, 2078209.92it/s]


In [49]:
from symspellpy.symspellpy import SymSpell, Verbosity

# Initialize with a maximum edit distance and a prefix length
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)

# Load a dictionary file (this file contains words and their frequencies)
dictionary_path = "symspellpy/frequency_dictionary_en_82_765.txt"
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

True

In [50]:
corrections = {}
misspellings_found=0
for word in tqdm.tqdm(missed_words):
    suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=1)
    if len(suggestions)!=0 and suggestions[0].term!= word :
        corrections[word] = suggestions[0].term
        misspellings_found += 1

100%|██████████| 375112/375112 [00:09<00:00, 38609.97it/s]


In [51]:
missed_words_uncategorized_corrected = {corrections.get(word, word) for word in missed_words}

In [52]:
translator_with_corrections = load_vectors(r"..\embeddings\fasttext_data\wiki-news-300d-1M.vec", set(corrections.values()))

In [53]:
# lemmatize
import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\belas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\belas\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [54]:
from nltk.corpus import wordnet

def get_wordnet_pos(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [55]:
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

lemmatizer = WordNetLemmatizer()

tagged_words = pos_tag(list(real_words.union(missed_words_uncategorized_corrected)))

lemmatized_words = {}
for word, tag in tqdm.tqdm(tagged_words):
    wordnet_tag = get_wordnet_pos(tag)
    lemma = lemmatizer.lemmatize(word, pos=wordnet_tag)
    lemmatized_words[word] = lemma




100%|██████████| 291408/291408 [00:02<00:00, 136516.20it/s]


In [56]:
translator_with_lemmatization = load_vectors(r"..\embeddings\fasttext_data\wiki-news-300d-1M.vec", set(lemmatized_words.values()))

In [68]:
# calculate final capture over unique words
immediately_recognized_words = 0
words_recognized_after_spellcheck = 0
words_recognized_after_lemmatization = 0
totally_unrecognized_words = 0
unrecognized_words = set()
for word in tqdm.tqdm(unique_words):
    if word in translator:
        immediately_recognized_words += 1
    elif word in corrections and corrections[word] in translator_with_corrections:
        words_recognized_after_spellcheck += 1
    elif (lemmatized_words.get(corrections.get(word, word), word) in translator_with_lemmatization) or (lemmatized_words.get(word, word) in translator_with_lemmatization):
        words_recognized_after_lemmatization += 1
    else:
        totally_unrecognized_words += 1
        unrecognized_words.add(word)


100%|██████████| 480706/480706 [00:00<00:00, 641901.82it/s]


In [58]:
print(f"after spellcheck {words_recognized_after_spellcheck+immediately_recognized_words} are recognized. thats {(words_recognized_after_spellcheck+immediately_recognized_words)/len(unique_words)*100:.0f}%")


after spellcheck 215258 are recognized. thats 45%


In [66]:
print(f"immediately_recognized_words: {immediately_recognized_words}")
print(f"words_recognized_after_spellcheck: {words_recognized_after_spellcheck}")
print(f"words_recognized_after_lemmatization: {words_recognized_after_lemmatization}")
print(f"totally_unrecognized_words: {totally_unrecognized_words}")

immediately_recognized_words: 105594
words_recognized_after_spellcheck: 109664
words_recognized_after_lemmatization: 735
totally_unrecognized_words: 264713


In [ ]:
# compute absolute capture over corpus
known_words = 0
unknown_words = 0
for doc in tqdm.tqdm(cleaned_corpus):
    for word in doc:
        if word in unrecognized_words:
            unknown_words += 1
        else:
            known_words += 1

In [72]:

print(f"known words: {known_words}")
print(f"unknown words: {unknown_words}")
print(f"ratio: {known_words/(known_words+unknown_words)*100:.2f}%")

known words: 298324268
unknown words: 3330539
ratio: 98.90%


In [73]:
words_recognized_after_lemmatization/len(unique_words)

0.0015290010942239124

In [80]:
# create the final translator and pickle save it
final_translator = translator.copy()
for word, corrected_word in corrections.items():
    if corrected_word in translator_with_corrections:
        final_translator[word] = translator_with_corrections[corrected_word]
    if word in lemmatized_words and lemmatized_words[word] in translator_with_lemmatization:
        final_translator[word] = translator_with_lemmatization[lemmatized_words[word]]
    if corrected_word in lemmatized_words and lemmatized_words[corrected_word] in translator_with_lemmatization:
        final_translator[word] = translator_with_lemmatization[lemmatized_words[corrected_word]]

for missed_word in missed_words:
    if missed_word in lemmatized_words and lemmatized_words[missed_word] in translator_with_lemmatization:
        final_translator[missed_word] = translator_with_lemmatization[lemmatized_words[missed_word]]

In [81]:
import pickle
import os

filename = "vector_translator/translator.pickle"

os.makedirs(os.path.dirname(filename), exist_ok=True)
with open(filename, "wb") as f:
    pickle.dump(final_translator, f)

